In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import random
import numpy as np

import time


SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', lower = True)
LABEL = data.LabelField()

In [0]:
train_data, valid_data, test_data = datasets.SNLI.splits(TEXT, LABEL)


In [0]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 549367
Number of validation examples: 9842
Number of testing examples: 9824


In [0]:
print(vars(train_data.examples[0]))


{'premise': ['a', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane', '.'], 'hypothesis': ['a', 'person', 'is', 'training', 'his', 'horse', 'for', 'a', 'competition', '.'], 'label': 'neutral'}


In [0]:
MIN_FREQ = 2

TEXT.build_vocab(train_data, 
                 min_freq = MIN_FREQ,
                 vectors = "glove.6B.300d",
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [0]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")


Unique tokens in TEXT vocabulary: 23566


In [0]:
print(TEXT.vocab.freqs.most_common(20))


[('a', 1438991), ('.', 962558), ('the', 534692), ('in', 407296), ('is', 373543), ('man', 266236), ('on', 235904), ('and', 206363), ('are', 199114), ('of', 192428), ('with', 169236), ('woman', 137630), ('two', 122259), ('people', 121154), (',', 114331), ('to', 113972), ('at', 98656), ('wearing', 81024), ('an', 80212), ('his', 72467)]


In [0]:
print(TEXT.vocab.itos[:10])


['<unk>', '<pad>', 'a', '.', 'the', 'in', 'is', 'man', 'on', 'and']


In [0]:
print(LABEL.vocab.itos)


['entailment', 'contradiction', 'neutral']


In [0]:
print(LABEL.vocab.freqs.most_common())


[('entailment', 183416), ('contradiction', 183187), ('neutral', 182764)]


In [0]:
BATCH_SIZE = 512

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [0]:
class NLISum(nn.Module):
    def __init__(self, 
                 vocab_size, 
                 embedding_dim,
                 hidden_dim,
                 fc_layers,
                 output_dim, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.translation = nn.Linear(embedding_dim, hidden_dim)
        
        fcs = [nn.Linear(hidden_dim * 2, hidden_dim * 2) for _ in range(fc_layers)]
        
        self.fcs = nn.ModuleList(fcs)
        
        self.fc_out = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, prem, hypo):

        #prem = [prem sent len, batch size]
        #hypo = [hypo sent len, batch size]
        
        embedded_prem = self.embedding(prem)
        embedded_hypo = self.embedding(hypo)
        
        #embedded_prem = [prem sent len, batch size, embedding dim]
        #embedded_hypo = [hypo sent len, batch size, embedding dim]
        
        translated_prem = F.relu(self.translation(embedded_prem))
        translated_hypo = F.relu(self.translation(embedded_hypo))
        
        #translated_prem = [prem sent len, batch size, hidden dim]
        #translated_hypo = [hypo sent len, batch size, hidden dim]
        
        hidden_prem = translated_prem.sum(dim = 0)
        hidden_hypo = translated_hypo.sum(dim = 0)
            
        #hidden_x = [batch size, hid dim]
        
        hidden = torch.cat((hidden_prem, hidden_hypo), dim = 1)

        #hidden = [batch size, hid dim * 2]
            
        for fc in self.fcs:
            hidden = fc(hidden)
            hidden = F.relu(hidden)
            hidden = self.dropout(hidden)
        
        prediction = self.fc_out(hidden)
        
        #prediction = [batch size, output dim]
        
        return prediction




In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 300
FC_LAYERS = 3
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.25
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

MODEL='gru'

if MODEL=='linear':
  model = NLISum(INPUT_DIM,
               EMBEDDING_DIM,
               HIDDEN_DIM,
               FC_LAYERS,
               OUTPUT_DIM,
               DROPOUT,
               PAD_IDX)
elif MODEL=='gru':
  model = NLIRNN(INPUT_DIM,
                'gru',
                EMBEDDING_DIM,
                HIDDEN_DIM,
                FC_LAYERS,
                OUTPUT_DIM,
                DROPOUT,
                PAD_IDX)
elif MODEL=='lstm':
  model = NLIRNN(INPUT_DIM,
                'lstm',
                EMBEDDING_DIM,
                HIDDEN_DIM,
                FC_LAYERS,
                OUTPUT_DIM,
                DROPOUT,
                PAD_IDX)



In [0]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1)
        
model.apply(init_weights)

NLIRNN(
  (embedding): Embedding(23566, 300, padding_idx=1)
  (translation): Linear(in_features=300, out_features=300, bias=True)
  (rnn): GRU(300, 300)
  (fcs): ModuleList(
    (0): Linear(in_features=600, out_features=600, bias=True)
    (1): Linear(in_features=600, out_features=600, bias=True)
    (2): Linear(in_features=600, out_features=600, bias=True)
  )
  (fc_out): Linear(in_features=600, out_features=3, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 8,785,503 trainable parameters


In [0]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([23566, 300])


In [0]:
model.embedding.weight.data.copy_(pretrained_embeddings)


tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [-0.2971,  0.0940, -0.0967,  ...,  0.0597, -0.2285,  0.2960],
        ...,
        [-0.2149,  0.0846, -0.2949,  ...,  0.2379,  0.4804, -0.3348],
        [-0.2983, -0.2664, -0.0631,  ..., -0.1577,  1.0438, -0.6433],
        [ 0.3580, -0.0304,  0.3355,  ...,  0.0703, -0.5158,  0.1819]])

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2971,  0.0940, -0.0967,  ...,  0.0597, -0.2285,  0.2960],
        ...,
        [-0.2149,  0.0846, -0.2949,  ...,  0.2379,  0.4804, -0.3348],
        [-0.2983, -0.2664, -0.0631,  ..., -0.1577,  1.0438, -0.6433],
        [ 0.3580, -0.0304,  0.3355,  ...,  0.0703, -0.5158,  0.1819]])


In [0]:
model.embedding.weight.requires_grad = False


In [0]:
print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 1,715,703 trainable parameters


In [0]:
optimizer = optim.Adam(model.parameters(),lr=1e-4)


In [0]:
criterion = nn.CrossEntropyLoss()


In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        prem = batch.premise
        hypo = batch.hypothesis
        labels = batch.label
        
        optimizer.zero_grad()
        
        #prem = [prem sent len, batch size]
        #hypo = [hypo sent len, batch size]
        
        predictions = model(prem, hypo)
        
        #predictions = [batch size, output dim]
        #labels = [batch size]
        
        loss = criterion(predictions, labels)
                
        acc = categorical_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            prem = batch.premise
            hypo = batch.hypothesis
            labels = batch.label
                        
            predictions = model(prem, hypo)
            
            loss = criterion(predictions, labels)
                
            acc = categorical_accuracy(predictions, labels)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 6s
	Train Loss: 0.536 | Train Acc: 78.59%
	 Val. Loss: 0.751 |  Val. Acc: 67.75%
Epoch: 02 | Epoch Time: 1m 6s
	Train Loss: 0.529 | Train Acc: 78.96%
	 Val. Loss: 0.748 |  Val. Acc: 67.80%
Epoch: 03 | Epoch Time: 1m 8s
	Train Loss: 0.522 | Train Acc: 79.31%
	 Val. Loss: 0.739 |  Val. Acc: 68.49%
Epoch: 04 | Epoch Time: 1m 9s
	Train Loss: 0.514 | Train Acc: 79.63%
	 Val. Loss: 0.742 |  Val. Acc: 68.10%
Epoch: 05 | Epoch Time: 1m 10s
	Train Loss: 0.507 | Train Acc: 80.01%
	 Val. Loss: 0.748 |  Val. Acc: 67.72%
Epoch: 06 | Epoch Time: 1m 10s
	Train Loss: 0.501 | Train Acc: 80.33%
	 Val. Loss: 0.744 |  Val. Acc: 68.17%
Epoch: 07 | Epoch Time: 1m 10s
	Train Loss: 0.493 | Train Acc: 80.68%
	 Val. Loss: 0.754 |  Val. Acc: 67.75%
Epoch: 08 | Epoch Time: 1m 10s
	Train Loss: 0.487 | Train Acc: 81.00%
	 Val. Loss: 0.761 |  Val. Acc: 67.20%
Epoch: 09 | Epoch Time: 1m 11s
	Train Loss: 0.481 | Train Acc: 81.29%
	 Val. Loss: 0.755 |  Val. Acc: 67.98%
Epoch: 10 | Epoch Time:

In [0]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.751 |  Test Acc: 67.86%
